In [7]:
#ln -s /Users/giakhang/Downloads/indoor_CVPR_dataset .
!unzip /content/drive/MyDrive/dataset_zoo/Indoor_CVPR_dataset.zip -d .

Streaming output truncated to the last 5000 lines.
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport_inside_0417.jpg  
  inflating: ./__MACOSX/Indoor_CVPR_dataset/images/airport_inside/._airport_inside_0417.jpg  
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport_inside_0371.jpg  
  inflating: ./__MACOSX/Indoor_CVPR_dataset/images/airport_inside/._airport_inside_0371.jpg  
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport_inside_0416.jpg  
  inflating: ./__MACOSX/Indoor_CVPR_dataset/images/airport_inside/._airport_inside_0416.jpg  
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport_inside_0370.jpg  
  inflating: ./__MACOSX/Indoor_CVPR_dataset/images/airport_inside/._airport_inside_0370.jpg  
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport_inside_0364.jpg  
  inflating: ./__MACOSX/Indoor_CVPR_dataset/images/airport_inside/._airport_inside_0364.jpg  
  inflating: ./Indoor_CVPR_dataset/images/airport_inside/airport

In [3]:
import numpy as np
import progressbar
import imutils
import random
import cv2
import os
from pathlib import Path

In [9]:
def create_dataset():
    DATASET_BASE = Path('Indoor_CVPR_dataset')

    # 1. Get file paths
    image_paths = []
    for root, dirs, files in os.walk(DATASET_BASE / 'images'):
        for file in files:
            image_paths.append(os.path.join(root, file))

    # 2. Randomly select 10000 images
    random.shuffle(image_paths)
    image_paths = image_paths[:10000]

    # 3. Rotate images
    # {rotated_angle: num_of_imgs, ...}
    angles = {0: 0,
              90: 0,
              180: 0,
              270: 0}

    # Start processbar
    bar = progressbar.ProgressBar(maxval=len(image_paths))
    bar.start()

    for i, image_path in enumerate(image_paths):
        image = cv2.imread(image_path)
        angle = np.random.choice(list(angles.keys()))

        if image is None:
            continue

        # Rotate image
        image = imutils.rotate_bound(image, angle)

        # Output path
        base = os.path.sep.join([str(DATASET_BASE / 'rotated_images'), str(angle)])
        if not os.path.exists(base):
            os.makedirs(base)

        index = image_path.rfind(".")
        ext = image_path[index:]
        output_path = [base, "image_{}{}".format(str(angles.get(angle, 0)).zfill(5), ext)]
        output_path = os.path.sep.join(output_path)

        # Write file
        cv2.imwrite(output_path, image)

        angles[angle] = angles.get(angle, 0) + 1
        bar.update(i)

    bar.finish()

    for angle in list(angles.keys()):
        print('Số lượng ảnh góc {}: {}'.format(angle, angles[angle]))

In [10]:
create_dataset()

100% (10000 of 10000) |##################| Elapsed Time: 0:03:02 Time:  0:03:02


Số lượng ảnh góc 0: 2517
Số lượng ảnh góc 90: 2589
Số lượng ảnh góc 180: 2451
Số lượng ảnh góc 270: 2427


In [6]:
# import the necessary packages
import h5py
import os


class HDF5DatasetWriter:
    def __init__(self, dims, outputPath, dataKey='images', bufSize=1000):
        '''
        check to see if the output path exists, and if so, raise an exception
        '''
        if os.path.exists(outputPath):
            raise ValueError("The supplied 'output path' already exists and cannot be overwritten. Manually delete the file before contuinuing", outputPath)

        # open the HDF5 database for writing and create two datasets: one to store the images/feature and another to store class labels
        self.db = h5py.File(outputPath, 'w')
        self.data = self.db.create_dataset(dataKey, dims, "float")
        self.labels = self.db.create_dataset('labels', (dims[0],), int)

        # store the buffer size, then initizlize the buffer itself along with the index into the database
        self.bufSize = bufSize
        self.buffer = {'data': [], 'labels': []}
        self.idx = 0

    def add(self, rows, labels):
        '''
        add the rows and labels to the buffer
        '''
        self.buffer['data'].extend(rows)
        self.buffer['labels'].extend(labels)

        # check to see if the buffer needs to be flused to disk
        if len(self.buffer['data']) >= self.bufSize:
            self.flush()

    def flush(self):
        '''
        write the buffers to disk then reset the buffer
        '''
        i = self.idx + len(self.buffer['data'])
        self.data[self.idx:i] = self.buffer['data']
        self.labels[self.idx:i] = self.buffer['labels']
        self.idx = i
        self.buffer = {'data': [], 'labels': []}

    def storeClassLabels(self, classLabels):
        '''
        create a dataset to store the actual class label names, then store the class labels
        '''
        dt = h5py.special_dtype(vlen=str)
        labelSet = self.db.create_dataset('label_names', (len(classLabels),), dtype=dt)
        labelSet[:] = classLabels

    def close(self):
        '''
        check to see if ther is any other entries in the buffer that need to be flushed to the disk
        '''
        if len(self.buffer['data']) > 0:
            self.flush()

        # close the dataset
        self.db.close()

In [2]:
from sklearn.preprocessing import LabelEncoder
from imutils import paths
import numpy as np
import progressbar
import random
import os
from pathlib import Path
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical, img_to_array, load_img

def feature_extraction():
    DATASET_BASE = Path('Indoor_CVPR_dataset')

    # 1. Get rotated images' paths and their labels
    image_paths, labels = [], []
    rotated_images = DATASET_BASE / 'rotated_images'

    for root, dirs, files in os.walk(rotated_images):
        for file in files:
            if file[-3:] in ['jpg', 'png', 'jpeg']:
                image_paths.append(os.path.join(root, file))

                # Chỉ dùng cho Linux hoặc MacOS,
                labels.append(int(root.split('/')[-1]))

    # 2. Encode label to number
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # 3. Load VGG16
    model = VGG16(weights='imagenet', include_top=False)

    # 4. Create HDF5 database
    hdf5_file = DATASET_BASE / 'hdf5' / 'features.hdf5'
    if not os.path.exists(DATASET_BASE / 'hdf5'):
        os.makedirs(DATASET_BASE / 'hdf5')
    if os.path.exists(hdf5_file):
        os.remove(hdf5_file)
    dataset = HDF5DatasetWriter((len(image_paths), 512 * 7 * 7),
                                hdf5_file, dataKey="features", bufSize=1000)

    # 5. Save label names to datset
    dataset.storeClassLabels([str(cls) for cls in le.classes_])

    bar = progressbar.ProgressBar(maxval=len(image_paths))
    bar.start()

    # 6. Iterate over batches
    bs = 32
    for i in np.arange(0, len(image_paths), bs):
        # Get image path and its label
        batch_paths = image_paths[i:i + bs]
        batch_labels = labels[i:i + bs]

        # Batch images
        batch_images = []
        for j, image_path in enumerate(batch_paths):
            # Load image
            target_size = (224, 224)
            image = load_img(image_path, color_mode='rgb', target_size=target_size)
            image = img_to_array(image)

            # Preprocess image
            image = np.expand_dims(image, 0)
            image = preprocess_input(image)

            # Add to batches
            batch_images.append(image)

        # Get features from batch
        batch_images = np.vstack(batch_images)
        features = model.predict(batch_images, verbose=0)

        # Reshape features
        features = features.reshape((features.shape[0], 512 * 7 * 7))

        assert features.shape == (features.shape[0], 512 * 7 * 7)
        assert features.shape[0] == len(batch_labels)

        # Add features labels to HDF5 dataset
        dataset.add(features, batch_labels)
        bar.update(i)

    dataset.close()
    bar.finish()

In [3]:
feature_extraction()

100% |#########################################################################|


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py

def build_model(input_shape, num_classes):
    model = Sequential([
        Dense(1024, activation='relu'),
        Dropout(0.3),

        Dense(512, activation='relu'),
        Dropout(0.3),

        Dense(num_classes, activation='softmax')
    ])
    return model

def train_model():
    # Load data from HDF5 file
    with h5py.File('Indoor_CVPR_dataset/hdf5/features.hdf5', 'r') as f:
        X_train = f['features'][:]
        y_train = f['labels'][:]
        num_cls = len(f['label_names'])

    # Build model
    model = build_model(X_train.shape[1], num_cls)

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Define callbacks, like model checkpoint to save the best model during training
    checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

    # Train the model
    history = model.fit(X_train, y_train,
                        batch_size=8,
                        epochs=100,
                        validation_split=0.1,
                        callbacks=[checkpoint_callback],
                        verbose=1)

    return history

In [7]:
train_model()

Epoch 1/500
1121/1123 [============================>.] - ETA: 0s - loss: 5.2487 - accuracy: 0.7506

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1123/1123 [==============================] - 19s 16ms/step - loss: 5.2414 - accuracy: 0.7506 - val_loss: 0.9601 - val_accuracy: 0.6583
Epoch 2/500
1123/1123 [==============================] - 14s 13ms/step - loss: 0.6287 - accuracy: 0.8596 - val_loss: 0.8186 - val_accuracy: 0.6914
Epoch 3/500
1123/1123 [==============================] - 17s 15ms/step - loss: 0.4439 - accuracy: 0.8873 - val_loss: 0.7578 - val_accuracy: 0.7124
Epoch 4/500
1123/1123 [==============================] - 14s 13ms/step - loss: 0.3427 - accuracy: 0.9149 - val_loss: 0.8236 - val_accuracy: 0.7565
Epoch 5/500
1123/1123 [==============================] - 11s 10ms/step - loss: 0.4181 - accuracy: 0.9031 - val_loss: 1.8014 - val_accuracy: 0.6633
Epoch 6/500
1123/1123 [==============================] - 14s 12ms/step - loss: 0.3982 - accuracy: 0.9088 - val_loss: 0.8806 - val_accuracy: 0.7836
Epoch 7/500
1123/1123 [==============================] - 14s 13ms/step - loss: 0.3528 - accuracy: 0.9109 - val_loss: 0.5525 - val_

KeyboardInterrupt: 

In [4]:
from keras.models import load_model
from pathlib import Path
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical, img_to_array, load_img
import numpy as np
from tensorflow.keras.layers import Flatten
import tensorflow as tf
import imutils
import matplotlib.pyplot as plt

def valid_model():
    # 1. Load label names trong file hdf5
    with h5py.File('/content/Indoor_CVPR_dataset/hdf5/features.hdf5', 'r') as f:
        angles = f['label_names'][:].astype(np.int32)

    # 2. Load ảnh
    target_size = (224, 224)
    image = load_img('/content/Indoor_CVPR_dataset/rotated_images/270/image_00012.jpg',
                     color_mode='rgb', target_size=target_size)
    image = img_to_array(image, dtype='uint8')
    image_batch = np.expand_dims(image, 0)
    image_batch = preprocess_input(image_batch)


    # 3. Load mô hình VGG16
    vgg16 = VGG16(weights='imagenet', include_top=False)

    # 4. Tính feature extraction
    features = vgg16.predict(image_batch, verbose=1)

    # 5. Load model
    classification_model = load_model("./best_model.h5")

    # 6. Sử dụng model để dự đoán góc
    flatten_features = Flatten()(features)
    pred = classification_model(flatten_features)
    pred = tf.squeeze(pred)
    pred_cls_id = tf.math.argmax(pred)
    pred_angle = angles[pred_cls_id]

    # 7. Quay ảnh góc
    rotated_image = imutils.rotate_bound(image, -pred_angle)

    # 8. Hiện ảnh chưa xoay và đã xoay
    fig, axes = plt.subplots(1, 2)

    axes[0].imshow(image)
    axes[0].set_title('Before rotating')

    axes[1].imshow(rotated_image)
    axes[1].set_title('After rotating')

    axes[0].axis('off')
    axes[1].axis('off')
    plt.suptitle('Predicted angle: {}'.format(pred_angle))

    plt.show()

In [5]:
valid_model()

NameError: name 'h5py' is not defined